In [37]:
!sudo pip3 install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [39]:
from google.colab import files
uploaded = files.upload()

Saving intents.json to intents (2).json


In [40]:
data = next(iter(uploaded.values()))
d = json.loads(data.decode())

In [41]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in d['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)
num_classes

8

In [42]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)
training_labels

array([4, 4, 4, 4, 4, 3, 3, 3, 7, 7, 7, 7, 0, 0, 0, 6, 6, 6, 5, 5, 5, 5,
       5, 5, 5, 2, 2, 2, 2, 2, 1, 1, 1])

In [43]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [44]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_3 (Dense)             (None, 16)                272       
                                                                 
 dense_4 (Dense)             (None, 16)                272       
                                                                 
 dense_5 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
__________________________________________________

In [ ]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

In [46]:
from os import truncate
def chat():
  # model = model
  # tokenizer = tokenizer
  # label encoder = lbl_encoder
  # max_len = max_len

  while True:
    inp = input("User: ")
    if inp.lower() == 'quit':
      break
    
    result = model.predict(
        keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                                   truncating='post', maxlen=max_len))
    tag = lbl_encoder.inverse_transform([np.argmax(result)])

    for i in d['intents']:
      if i['tag'] == tag:
        print("ChatBot: ", np.random.choice(i['responses']))

In [47]:
print("Start typing to chat with the bot (type quit to stop\n\n")
chat()

Start typing to chat with the bot (type quit to stop


User: Hello
ChatBot:  Hi there
User: Who are you?
ChatBot:  I'm OdooBot, an Artificial Intelligent bot
User: Good
ChatBot:  You're most welcome!
User: I have a problem
ChatBot:  Please provide us your complaint in order to assist you
User: umm, how do I create an account?
ChatBot:  You can just easily create a new account from our web site
User: Got it, thanks
ChatBot:  Happy to help!
User: quit
